In [1]:
# Required imports.
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

import pandas as pd

In [2]:
AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base", "dis_severe", "dis_enhanced"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 100)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = ([AGE_BOUNDS] * len(AGE_CATEGORIES) +
          [DIS_BOUNDS] * len(DIS_CATEGORIES) +
          [GEO_BOUNDS] * len(REGIONS))

input_dict = {category: bound for category, bound in zip(categories, bounds)}
print(input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_base': (0, 100), 'dis_severe': (0, 100), 'dis_enhanced': (0, 100), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [3]:
def opt(reform):
    return optimize(input_dict, "mean_pct_loss", reform,
                    verbose=False, maxiter=None, seed=0,
                    # reform_3 > reform_2 with default of 0.01, or 0.005.
                    tol=0.001)

In [4]:
%%time
reform_1 = opt("reform_1")
print(reform_1[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.515000e-01
losses                1.401599e+11
mean_pct_loss         5.190000e-02
mean_pct_loss_pwd2    5.910000e-02
gini                  2.749000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0519 

Optimal solution:
 adult               124
senior              205
child                76
dis_base              0
dis_severe            0
dis_enhanced          0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.05193291339399926
 message: 'Optimization terminated successfully.'
    nfev: 2841
     nit: 10
 success: True
       x: array([205.22258067,  75.59287535,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.      

In [5]:
%%time
reform_2 = opt("reform_2")
print(reform_2[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.530000e-01
losses                1.420697e+11
mean_pct_loss         4.960000e-02
mean_pct_loss_pwd2    5.330000e-02
gini                  2.702000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0496 

Optimal solution:
 adult               119
senior              204
child                78
dis_base             95
dis_severe            1
dis_enhanced          1
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.04964600533389416
 message: 'Optimization terminated successfully.'
    nfev: 4881
     nit: 18
 success: True
       x: array([204.42660757,  77.6707806 ,  95.11091214,   1.29386508,
         1.48564273,   0.        ,   0.        ,   0.        ,
         0.        ,   0.      

In [6]:
%%time
reform_3 = opt("reform_3")
print(reform_3[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.536000e-01
losses                1.424248e+11
mean_pct_loss         4.960000e-02
mean_pct_loss_pwd2    5.310000e-02
gini                  2.699000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0496 

Optimal solution:
 adult               113
senior              200
child                73
dis_base             98
dis_severe            3
dis_enhanced          0
NORTH_EAST            7
NORTH_WEST            9
YORKSHIRE             0
EAST_MIDLANDS         7
WEST_MIDLANDS         6
EAST_OF_ENGLAND       2
LONDON                8
SOUTH_EAST            1
SOUTH_WEST           10
WALES                 4
SCOTLAND              6
NORTHERN_IRELAND      9
dtype: int64
     fun: 0.049608217299912764
 message: 'Optimization terminated successfully.'
    nfev: 16101
     nit: 62
 success: True
       x: array([208.74918757,  82.18154776,  97.84394976,   2.86254812,
         0.45167836,  -2.40476102,   0.27217166,  -8.95768311,
        -1.86305005,  -2.6251

Check that iterations improve.

In [7]:
assert reform_2[0].fun < reform_1[0].fun, "Reform 2 doesn't improve on 1"
assert reform_3[0].fun < reform_2[0].fun, "Reform 3 doesn't improve on 2"

Construct DataFrame to export, with parameter and optimization values.

In [8]:
ubi_params = pd.DataFrame(reform_1[1]).T
ubi_params.loc[1] = reform_2[1]
ubi_params.loc[2] = reform_3[1]
ubi_params['mean_pct_loss'] = [reform_1[0].fun,
                               reform_2[0].fun,
                               reform_3[0].fun]
ubi_params

,adult,senior,child,dis_base,dis_severe,dis_enhanced,NORTH_EAST,NORTH_WEST,YORKSHIRE,EAST_MIDLANDS,WEST_MIDLANDS,EAST_OF_ENGLAND,LONDON,SOUTH_EAST,SOUTH_WEST,WALES,SCOTLAND,NORTHERN_IRELAND,mean_pct_loss
0,123.922887,205.222581,75.592875,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.051933
1,119.440596,204.426608,77.670781,95.110912,1.293865,1.485643,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.049646
2,113.114604,199.791504,73.223865,97.843950,2.862548,0.451678,6.552922,9.229855,0.0,7.094633,6.332561,1.604237,8.017837,1.471345,9.862452,3.971185,6.13891,8.957683,0.049608


In [9]:
ubi_params.to_csv("optimal_params.csv", index=False)